#### Unnecessary - Copies just Repo records from one database to another
##### It IS a good example of batch queries and inserts

In [ ]:
import sys
import mariadb
import time
import hashlib
local_lib_dir = '../../../python/lib/'
sys.path.append(local_lib_dir)
from kitchen_sink_class import load_single_line_from_file as load_it
pwd = load_it('./db.pwd')

this_is_necessary = False

if this_is_necessary:
    mydb = mariadb.connect(
        port=3306,
        host='127.0.0.1',
        user='w3hacknet',
        password=pwd,
        database='w3hacknet',
        autocommit=True
    )
    dest_db = mariadb.connect(
        port=13306,
        host='127.0.0.1',
        user='w3hacknet',
        password=pwd,
        database='w3hacknet',
        autocommit=False
    )

    cursor = mydb.cursor()
    dest_cursor = dest_db.cursor()
    cursor.execute('select owner, name, commit_count, min_date, max_date from repo;')
    running = True
    rec_count = 0
    while running:
        rslt_set = cursor.fetchmany(size=100)
        if rslt_set:
            batch = []
            for (owner, name, commit_count, min_date, max_date) in rslt_set:
                batch.append((owner, name, commit_count, min_date, max_date))
            dest_cursor.executemany('insert ignore into w3hacknet.repo (owner, name, commit_count, min_date, max_date) values (?, ?, ?, ?, ?)', batch)
            dest_db.commit()
            rec_count += len(batch)
            print(rec_count,'records copied')
        else:
            running = False
    cursor.close()
    dest_cursor.close()

#### Better way to clone a database, procedures included
##### On the From database command line:
`mysqldump -p w3hacknet --routines > dump_of_matts_db.sql`
##### On the To database command line:
`mysql -p w3hacknet <dump_of_matts_db.sql`